#Airbnb Listing Data Cleansing

In [ ]:
import pandas as pd
import re
from google.colab import drive
from google.colab import files 

<br />

## Part 1: Import Data

We first import the data from Kaggle using an API. 

In [ ]:
# Step 1: install dependencies 
!pip install -q kaggle 

# Step 2: create API Token from Kaggle and upload Kaggle.json 
# Instructions : https://www.kaggle.com/general/74235
files.upload()

In [ ]:
# Step 3: additional steps 
!mkdir ~/.kaggle 
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Step 4: download the dataset 
# https://www.kaggle.com/samyukthamurali/airbnb-ratings-dataset
!kaggle datasets download -d samyukthamurali/airbnb-ratings-dataset

# Step 5: Unzip the files 
!unzip airbnb-ratings-dataset.zip

 99% 1.27G/1.27G [00:15<00:00, 75.6MB/s]
100% 1.27G/1.27G [00:15<00:00, 90.4MB/s]
Archive:  airbnb-ratings-dataset.zip
  inflating: LA_Listings.csv         
  inflating: NY_Listings.csv         
  inflating: airbnb-reviews.csv      
  inflating: airbnb_ratings_new.csv  


In [ ]:
# Step 6: import the necessary data 

leasing_df = pd.read_csv("/content/NY_Listings.csv", encoding='latin-1')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (7,19,24) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<br />

## Part 2: Initial Cleansing

Begin by removing columns that we do not need. 

In [ ]:
leasing_df.drop(columns = ['Host Response Rate', 'Street', 'State', 'Country', 'Property type', \
                           'Availability 365', 'Calendar last scraped', 'Last Review Date', 'Reviews per month'],
                inplace = True)

In [ ]:
# total score above 100 (max 100) makes no sense 
leasing_df = leasing_df[leasing_df["Review Scores Rating"] <= 100]

# capitalize 
leasing_df['City'] = leasing_df['City'].apply(lambda x : x.upper())
leasing_df['Neighbourhood cleansed'] = leasing_df['Neighbourhood cleansed'].apply(lambda x : x.upper())
leasing_df['Room type'] = leasing_df['Room type'].apply(lambda x : x.upper())


View what we have.

In [ ]:
leasing_df

,Listing ID,Name,Host ID,Host Name,Host Is Superhost,Host total listings count,City,Neighbourhood cleansed,latitude,longitude,Room type,Accommodates,Bathrooms,Bedrooms,Amenities,Price,Minimum nights,Maximum nights,Number of reviews,Review Scores Rating,Review Scores Accuracy,Review Scores Cleanliness,Review Scores Checkin,Review Scores Communication,Review Scores Location,Review Scores Value
0,2515,Sunny Private Room,16286162,Pat,False,4.0,BRONX,ALLERTON,40.866889,-73.857756,PRIVATE ROOM,1.0,1.0,1.0,Cable TV;Internet;Wireless Internet;Kitchen;Fr...,43,2,30.0,66,96,10,9,10,10,9,10
1,2539,Comfy bedroom minutes to Manhattan,44260966,Alicia,False,1.0,BRONX,SOUNDVIEW,40.829392,-73.865137,PRIVATE ROOM,1.0,1.0,1.0,TV;Internet;Wireless Internet;Air conditioning...,28,2,31.0,38,89,10,9,9,10,9,9
2,2595,Spacious RM in Private Home; Backyard/Near Metro,105394139,Ciprian,False,16.0,BRONX,FORDHAM,40.869139,-73.895096,PRIVATE ROOM,4.0,3.0,2.0,Internet;Wireless Internet;Air conditioning;Ki...,80,3,31.0,18,90,9,9,10,9,9,9
3,3330,Modern Bronx Treasure!,104262517,Felicia,False,1.0,BRONX,FORDHAM,40.868719,-73.891438,ENTIRE HOME/APT,4.0,1.0,1.0,TV;Internet;Wireless Internet;Air conditioning...,140,2,1125.0,7,85,9,10,8,9,9,9
4,3647,Good Vibes Sanctuary,58126473,Miriam,False,1.0,BRONX,FORDHAM,40.863628,-73.894787,PRIVATE ROOM,2.0,1.0,1.0,TV;Wireless Internet;Air conditioning;Kitchen;...,60,1,1125.0,56,95,10,10,10,10,9,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75744,42806780,Male share room long term,305126403,Bruce,False,3.0,BROOKLYN,FLATBUSH,40.641480,-73.960730,SHARED ROOM,NaN,NaN,NaN,NaN,22,30,NaN,0,0,0,0,0,0,0,0
75745,42815083,Jfk flight attendant crashpad,63729602,Adriel,False,1.0,QUEENS,SPRINGFIELD GARDENS,40.681430,-73.754610,SHARED ROOM,NaN,NaN,NaN,NaN,50,1,NaN,0,0,0,0,0,0,0,0
75746,42831981,Double-Bed in Sunset Park/Industry City,264569855,Leidi,False,3.0,BROOKLYN,SUNSET PARK,40.647210,-74.014180,SHARED ROOM,NaN,NaN,NaN,NaN,45,1,NaN,0,0,0,0,0,0,0,0
75747,42862086,(B6) BedStuy/Bushwick Shared Basement,538108,Kevin,False,6.0,BROOKLYN,BEDFORD-STUYVESANT,40.691970,-73.930030,SHARED ROOM,NaN,NaN,NaN,NaN,20,3,NaN,0,0,0,0,0,0,0,0


We now break the table into different parts. 

  1. Host
  2. Listing
  3. Neighborhood 
  4. Coordinate 
  5. Borough
  6. Amenities 
  7. Ratings 

In [ ]:
host_df = leasing_df[["Host ID", "Host Name", "Host total listings count"]]
neighborhood_df = leasing_df[["Neighbourhood cleansed", "City"]]
coordinate_df = leasing_df[["latitude", "longitude"]]
listing_df = leasing_df[["Host ID", "Listing ID", "Name", "Host Is Superhost", "Room type", "Accommodates", \
                         "Price", "Minimum nights", "Maximum nights", "latitude", "longitude", "City", "Neighbourhood cleansed"]]
ratings_df = leasing_df[["Host ID", "Listing ID", "Number of reviews", "Review Scores Rating", "Review Scores Accuracy", "Review Scores Cleanliness", \
                         "Review Scores Checkin", "Review Scores Communication", "Review Scores Location", "Review Scores Value"]]
amenities_df = leasing_df[["Host ID", "Listing ID", "Amenities", "Bathrooms", "Bedrooms"]]


<br />

## Part 3: Cleanse Host, Neighborhood, Coordinate Table

Host dataframe has non-empty primary keys, host name need not be modified, host total listings count properly represents null values as empty. 

Cleanse the neighborhood dataframe. We only need to drop duplicates as there are no invalid values.

In [ ]:
# drop duplicates 
neighborhood_df = neighborhood_df.drop_duplicates()

Coordinate dataframe has non-empty primary keys, and city has no invalid values. 

<br />

## Part 4: Cleanse Listing, Rating Table 

In [ ]:
# check for data types 
listing_df.dtypes

Host ID                     int64
Listing ID                  int64
Name                       object
Host Is Superhost            bool
Room type                  object
Accommodates              float64
Price                       int64
Minimum nights              int64
Maximum nights            float64
latitude                  float64
longitude                 float64
City                       object
Neighbourhood cleansed     object
dtype: object

In [ ]:
# check for data types 
ratings_df.dtypes

Host ID                        int64
Listing ID                     int64
Number of reviews              int64
Review Scores Rating           int64
Review Scores Accuracy         int64
Review Scores Cleanliness      int64
Review Scores Checkin          int64
Review Scores Communication    int64
Review Scores Location         int64
Review Scores Value            int64
dtype: object

In [ ]:
# check that null values are not hard coded, this should show as "NaN" in Pandas 
listing_df[pd.isna(listing_df['Accommodates']) == True]

# check that null values are not hard coded, this should show as "NaN" in Pandas 
listing_df[pd.isna(listing_df['Maximum nights']) == True]

,Host ID,Listing ID,Name,Host Is Superhost,Room type,Accommodates,Price,Minimum nights,Maximum nights,latitude,longitude,City,Neighbourhood cleansed
44310,8380,5552,Spacious river view in the West Village,False,ENTIRE HOME/APT,NaN,160,3,NaN,40.735520,-74.010420,MANHATTAN,WEST VILLAGE
44311,20950,7726,Hip Historic Brownstone Apartment with Backyard,False,ENTIRE HOME/APT,NaN,99,3,NaN,40.675920,-73.946940,BROOKLYN,CROWN HEIGHTS
44312,17985,7750,Huge 2 BR Upper East Cental Park,False,ENTIRE HOME/APT,NaN,190,7,NaN,40.796850,-73.948720,MANHATTAN,EAST HARLEM
44313,47727,12343,BEST BET IN HARLEM,False,ENTIRE HOME/APT,NaN,150,7,NaN,40.811750,-73.944780,MANHATTAN,HARLEM
44314,56094,14287,Cozy 1BD on Central Park West in New York City,False,ENTIRE HOME/APT,NaN,151,4,NaN,40.786350,-73.970080,MANHATTAN,UPPER WEST SIDE
...,...,...,...,...,...,...,...,...,...,...,...,...,...
75744,305126403,42806780,Male share room long term,False,SHARED ROOM,NaN,22,30,NaN,40.641480,-73.960730,BROOKLYN,FLATBUSH
75745,63729602,42815083,Jfk flight attendant crashpad,False,SHARED ROOM,NaN,50,1,NaN,40.681430,-73.754610,QUEENS,SPRINGFIELD GARDENS
75746,264569855,42831981,Double-Bed in Sunset Park/Industry City,False,SHARED ROOM,NaN,45,1,NaN,40.647210,-74.014180,BROOKLYN,SUNSET PARK
75747,538108,42862086,(B6) BedStuy/Bushwick Shared Basement,False,SHARED ROOM,NaN,20,3,NaN,40.691970,-73.930030,BROOKLYN,BEDFORD-STUYVESANT


<br />

## Part 5: Cleanse Amenities Table 

For "Bathrooms" and "Bedrooms", both hard coded NAs and blanks exist together in the original csv file. We check if both of these are represented as NaN.

In [ ]:
# check blank values are represented as NaN
amenities_df[amenities_df["Host ID"] == 2001830]

# check NA values are represented as NaN
amenities_df[amenities_df["Host ID"] == 48608026]

,Host ID,Listing ID,Amenities,Bathrooms,Bedrooms
62319,48608026,20663997,NaN,NaN,NaN


The "Amenities" column is an array of strings. We want to break this up into different columns. Not all values will be used, just the popular ones. 

The popular values are: kitchen, cable_tv, internet, wifi, free_parking, heating, air_conditioning. These values will be represented as boolean columns. 

In [ ]:
# first add the columns to the amenities dataframe 
amenities_df = amenities_df.assign(kitchen = [False for _ in range(len(amenities_df))],
                    cable_tv = [False for _ in range(len(amenities_df))],
                    internet = [False for _ in range(len(amenities_df))],
                    wifi = [False for _ in range(len(amenities_df))],
                    free_parking = [False for _ in range(len(amenities_df))],
                    heating = [False for _ in range(len(amenities_df))],
                    air_conditioning = [False for _ in range(len(amenities_df))])

amenities_df["Amenities"].fillna("[]", inplace = True)

def str_parser(strs):
  split_strs = strs.split(";")
  return split_strs

def fill_boolean_columns():
  for i in range(len(amenities_df)):
    pass 

amenities_df["Amenities"] = amenities_df['Amenities'].apply(lambda x : str_parser(x))


In [ ]:
for i in range(len(amenities_df)):
  for word in amenities_df['Amenities'].iat[i]:
    if word == "Cable TV":
      amenities_df["cable_tv"].iat[i] = True 
    if word == "Wireless Internet":
      amenities_df["wifi"].iat[i] = True
    if "free parking" in word.lower():
      amenities_df["free_parking"].iat[i] = True
    if word == "Air conditioning":
      amenities_df["air_conditioning"].iat[i] = True
    if word == "Kitchen" or word == "Internet" or word == "Heating":
      amenities_df[word.lower()].iat[i] = True
    
amenities_df.drop(columns=['Amenities'], inplace=True)

<br />

## Part 6: Rename Columns

In [ ]:
neighborhood_df = neighborhood_df.rename(columns={"Neighbourhood cleansed": "neighborhood", "City": "borough"})
neighborhood_df.reset_index(drop=True, inplace=True)

In [ ]:
host_df = host_df.rename(columns={"Host ID": "id", "Host Name": "hostname", "Host total listings count": "host_listing_count"})
host_df.reset_index(drop=True, inplace=True)

In [ ]:
coordinate_df.reset_index(drop=True, inplace=True)

In [ ]:
listing_df = listing_df.rename(columns={"Host ID": "host_id", "Listing ID": "listing_id", "Name": "listing_name",
                                        "Host Is Superhost": "host_is_superhost", "Room type": "room_type", 
                                        "Accommodates": "accommodates", "Price": "price", "Minimum nights": "min_nights", "Maximum nights": "max_nights", 
                                        "City": "borough", "Neighbourhood cleansed": "neighborhood"})
listing_df.reset_index(drop=True, inplace=True)

In [ ]:
ratings_df = ratings_df.rename(columns={"Host ID": "host_id", "Listing ID": "listing_id", "Number of reviews": "number_of_reviews",
                                        "Review Scores Rating": "rating_scores", "Review Scores Accuracy": "accuracy_score", 
                                        "Review Scores Cleanliness": "cleanliness_score", "Review Scores Checkin": "checkin_score", 
                                        "Review Scores Communication": "communication_score", "Review Scores Location": "location_score",
                                        "Review Scores Value": "value_score"})
ratings_df.reset_index(drop=True, inplace=True)

In [ ]:
amenities_df = amenities_df.rename(columns={"Host ID": "host_id", "Listing ID": "listing_id", "Bathrooms": "bathrooms", "Bedrooms": "bedrooms"})
amenities_df.reset_index(drop=True, inplace=True)

<br />

## Part 7: Export CSV files 

In [ ]:
# Mount your Drive to the Colab VM.
drive.mount('/gdrive')

# Write the DataFrame to CSV file.
with open('/gdrive/My Drive/neighborhood.csv', 'w') as f:
  neighborhood_df.to_csv(f)

with open('/gdrive/My Drive/host.csv', 'w') as f:
  host_df.to_csv(f)

with open('/gdrive/My Drive/coordinate.csv', 'w') as f:
  coordinate_df.to_csv(f)

with open('/gdrive/My Drive/listing.csv', 'w') as f:
  listing_df.to_csv(f)

with open('/gdrive/My Drive/ratings.csv', 'w') as f:
  ratings_df.to_csv(f)

with open('/gdrive/My Drive/amenities.csv', 'w') as f:
  amenities_df.to_csv(f)

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
